In [1]:
from datetime import datetime
import os
import json
from functools import partial




import re
from collections import Counter
import findspark
findspark.init()
import numpy as np
import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
from pyspark.sql import functions as F

In [2]:
index2amazon_column = {
    1: "customer_id",
    14: "review_date",
    6: "product_category",
    7: "star_rating"
}

basepath = "/Users/cedricpeeters/Documents/UA 2019-2020/Data Engineering/Opdracht/amazon"

In [4]:
amazon_rdd = sc.textFile(basepath)

In [5]:
amazon_rdd.take(1)

['marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date']

In [8]:
amazon_noheader = amazon_rdd.zipWithIndex().filter(lambda row_index: row_index[1] > 0).keys()

In [9]:
amazon_noheader.take(1)

['US\t53096384\tR63J84G1LOX6R\t1563890119\t763187671\tThe Sandman Vol. 1: Preludes and Nocturnes\tBooks\t4\t0\t1\tN\tN\tignore the review below\tthis is the first 8 issues of the series. it is the starting point of all this... it also contains the sound of her wings. issue #8. which is the first appearance of death. and many peoples favorite issue. its not the best of the collected works. but its the start of them.\t1995-08-13']

In [10]:
tab_split_func = lambda input_line: input_line.split("\t")

In [11]:
amazon_split_rdd = amazon_noheader.map(tab_split_func)

In [12]:
amazon_split_rdd.take(1)

[['US',
  '53096384',
  'R63J84G1LOX6R',
  '1563890119',
  '763187671',
  'The Sandman Vol. 1: Preludes and Nocturnes',
  'Books',
  '4',
  '0',
  '1',
  'N',
  'N',
  'ignore the review below',
  'this is the first 8 issues of the series. it is the starting point of all this... it also contains the sound of her wings. issue #8. which is the first appearance of death. and many peoples favorite issue. its not the best of the collected works. but its the start of them.',
  '1995-08-13']]

In [13]:
def transform_to_json(record, index2column):
    return_value = dict()
    for index, column_name in index2column.items():
        return_value[column_name] = record[index]
    return return_value
      
    
amazon_json_rdd = amazon_split_rdd.map(partial(transform_to_json, index2column=index2amazon_column))

In [14]:
amazon_json_rdd.take(2)

[{'customer_id': '53096384',
  'review_date': '1995-08-13',
  'product_category': 'Books',
  'star_rating': '4'},
 {'customer_id': '53096399',
  'review_date': '1995-08-17',
  'product_category': 'Books',
  'star_rating': '4'}]

In [15]:
def convert_dtstr_dt(dtstr):
    try:
        format_string = "%Y-%m-%d"
        dt = datetime.strptime(dtstr, format_string)
        return dt
    except ValueError:
        return None

def convert_to_integer(intstr):
    try:
        return int(intstr)
    except ValueError:
        return None

type_converters = {
    "review_date": convert_dtstr_dt,
    "star_rating": convert_to_integer
}

def convert_types(record, converters):
    for col_name, convert_func in converters.items():
        record[col_name] = convert_func(record[col_name])
    return record

In [16]:
amazon_converted_rdd = amazon_json_rdd.map(partial(convert_types, converters=type_converters))

In [17]:
amazon_converted_rdd.take(4)

[{'customer_id': '53096384',
  'review_date': datetime.datetime(1995, 8, 13, 0, 0),
  'product_category': 'Books',
  'star_rating': 4},
 {'customer_id': '53096399',
  'review_date': datetime.datetime(1995, 8, 17, 0, 0),
  'product_category': 'Books',
  'star_rating': 4},
 {'customer_id': '53096332',
  'review_date': datetime.datetime(1995, 8, 30, 0, 0),
  'product_category': 'Books',
  'star_rating': 5},
 {'customer_id': '53096335',
  'review_date': datetime.datetime(1995, 9, 11, 0, 0),
  'product_category': 'Books',
  'star_rating': 5}]

In [18]:
category_counts = amazon_converted_rdd.map(lambda js: (js["product_category"],1)).reduceByKey(lambda x, y: x+y)

In [19]:
category_counts.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 109 in stage 8.0 failed 1 times, most recent failure: Lost task 109.0 in stage 8.0 (TID 444, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/opt/apache-spark/libexec

In [20]:
largest_category = sorted(category_counts, key = lambda x: x[1], reverse=True)[0][0]

TypeError: 'PipelinedRDD' object is not iterable